## Setting up imports and paths

In [ ]:
import pandas as pd
import os
import json
from pathlib import Path
import re
import unicodedata
from typing import Optional
import importlib.metadata as m, json
ver = m.version("pybioclip")
print("pybioclip:", ver)


# --- Configuration ---
# Define the paths to your data
PROCESSED_DATA_DIR = Path('../data/processed/')
METADATA_PATH = '../data/raw/AI Image Data Sept 3 2025.csv'
IMAGE_DIR = Path("/Users/jayjivandas/Library/CloudStorage/OneDrive-TheOhioStateUniversity/Research-Onedrive/Imageomics/Bryant, Cece's files - Tick Images - Copy copy")
# Define JSON paths more explicitly
FULL_DATA_JSON = os.path.join(PROCESSED_DATA_DIR, "final_data.json")
FILTERED_DATA_JSON = os.path.join(PROCESSED_DATA_DIR, "final_data_no_ixodes.json")
CLASS_NAMES_JSON = os.path.join(PROCESSED_DATA_DIR, "class_names.json")
FILTERED_CLASS_NAMES_JSON = os.path.join(PROCESSED_DATA_DIR, "class_names_no_ixodes.json")


## Loading in the csv

In [39]:
# 1. Load the data from your CSV into a DataFrame.
df = pd.read_csv(METADATA_PATH)
df.fillna('', inplace=True)
# removing duplicates based on 'Sample ID'
df.drop_duplicates(subset=['Sample ID'], keep='first', inplace=True)

# 2. Get All Available Image Filenames
# Get a set of all available image filenames for fast lookups.
# We convert all filenames to uppercase to handle potential inconsistencies (e.g., .jpg vs .JPG).
all_image_files = {f.upper(): f for f in os.listdir(IMAGE_DIR)}

print(f"Loaded {len(df)} records from the metadata file.")
print(f"Found {len(all_image_files)} image files to search through.")

Loaded 665 records from the metadata file.
Found 1334 image files to search through.


/var/folders/tc/csl5ykwn1v36vwqf20swsn880000gn/T/ipykernel_67706/4201957005.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


## Determining the final list of valid ids
### cross referencing the csv specimens to that of the images 

In [40]:
# --- 3. Determine the Final List of Valid IDs ---
csv_ids = set(df['Sample ID'].astype(str)) # only unique IDs from the CSV
image_base_ids = {f.upper().rsplit('-', 1)[0] for f in all_image_files}
valid_ids = csv_ids.intersection(image_base_ids)

In [41]:
# --- 4. Filter DataFrame to Final Valid Set ---
final_df = df[df['Sample ID'].isin(valid_ids)].copy()
print(f"Validated. Proceeding to create JSON for {len(final_df)} specimens.")

Validated. Proceeding to create JSON for 664 specimens.


In [42]:
import re, unicodedata

def normalize_spaces(s):
    s = unicodedata.normalize("NFKC", s or "")
    s = "".join(" " if unicodedata.category(ch).startswith("Z") else ch for ch in s)  # fixes \u00A0
    s = "".join(ch for ch in s if unicodedata.category(ch) not in {"Cc","Cf"})        # drops zero-widths
    return re.sub(r"\s+", " ", s).strip()

DERM_VARIABLIS = re.compile(r"^\s*dermacentor\s+variablis\s*$", re.I)

def fix_label_minimal(raw):
    s = normalize_spaces(raw)
    if DERM_VARIABLIS.match(s):
        return "Dermacentor variabilis"
    parts = s.split()
    if len(parts) >= 2:
        return parts[0].capitalize() + " " + parts[1].lower()  # Genus species
    return s


In [43]:
final_df["true_label"] = final_df["Species of Tick"].astype(str).apply(fix_label_minimal)

# quick checks
print("NBSP left:", final_df["true_label"].str.contains("\u00A0", regex=False).sum())
print("'variablis' left:", (final_df["true_label"].str.lower() == "dermacentor variablis").sum())

# optional peek
# print(sorted(final_df["true_label"].unique()))
# print(final_df["true_label"].value_counts())


NBSP left: 0
'variablis' left: 0


## Building the JSON structure

In [44]:
# --- 5. Build the JSON Structure ---
final_data_list = []
class_names_set = set()
missing_ids = []

for index, row in final_df.iterrows():
    base_id = row['Sample ID'].upper()
    dorsal_path, ventral_path = None, None

    dorsal_patterns = [f"{base_id}-01.JPG", f"{base_id}-1.JPG"]
    ventral_patterns = [f"{base_id}-02.JPG", f"{base_id}-2.JPG"]

    for pattern in dorsal_patterns:
        if pattern in all_image_files:
            original_filename = all_image_files[pattern]
            dorsal_path = os.path.join(IMAGE_DIR, original_filename)
            break
    for pattern in ventral_patterns:
        if pattern in all_image_files:
            original_filename = all_image_files[pattern]
            ventral_path = os.path.join(IMAGE_DIR, original_filename)
            break

    label = row["true_label"]
    if dorsal_path and ventral_path:
        final_data_list.append({
            'image_path': dorsal_path,
            'true_label': label,
            'sample_id': row['Sample ID'],
            'view': 'dorsal',
            'sex': row.get('Tick Sex1'),
            'life_stage': row.get('Life Stage'),
            'attached': row.get('Attached?')
        })
        final_data_list.append({
            'image_path': ventral_path,
            'true_label': label,
            'sample_id': row['Sample ID'],
            'view': 'ventral',
            'sex': row.get('Tick Sex1'),
            'life_stage': row.get('Life Stage'),
            'attached': row.get('Attached?')
        })
        class_names_set.add(label)
    else:
        missing_ids.append(row['Sample ID'])

print(f"Created {len(final_data_list)} total entries for the JSON file.")
if missing_ids:
    print(f"Missing images for {len(missing_ids)} specimens: {missing_ids}")


Created 1324 total entries for the JSON file.
Missing images for 2 specimens: ['246-01', '33-01']


In [45]:
# --- 6. Save the JSON File ---
output_file = os.path.join(os.path.dirname(FULL_DATA_JSON), "final_data.json")
os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, 'w') as f:
    json.dump(final_data_list, f, indent=4)

print(f"Success! Data manifest saved to: {output_file}")

# Convert the class names set to a sorted list for consistent order
final_class_names = sorted(list(class_names_set))
with open(CLASS_NAMES_JSON, 'w') as f:
    json.dump(final_class_names, f, indent=4)

print(f"Success! Found {len(final_class_names)} unique class names and saved them to: {CLASS_NAMES_JSON}")

Success! Data manifest saved to: ../data/processed/final_data.json
Success! Found 8 unique class names and saved them to: ../data/processed/class_names.json


## Getting rid of all the entries with IXODES. Missing species

In [46]:
# --- 7. Create filtered JSON without single "Ixodes" entries ---
EXCLUDE = {"Ixodes"}  # Add variations of generic Ixodes

print("\n--- Starting Ixodes Filtering ---")
print(f"Found species before filtering: {sorted(list(class_names_set))}")

# Filter out generic Ixodes entries but keep Ixodes scapularis
filtered_data_list = [
    entry for entry in final_data_list 
    if entry['true_label'] not in EXCLUDE
]

# Save filtered data to new JSON
filtered_output_file = FILTERED_DATA_JSON  # Using predefined path
with open(filtered_output_file, 'w') as f:
    json.dump(filtered_data_list, f, indent=4)

# Create new class names list without generic Ixodes
filtered_class_names = [name for name in final_class_names if name not in EXCLUDE]
with open(FILTERED_CLASS_NAMES_JSON, 'w') as f:  # Using predefined path
    json.dump(filtered_class_names, f, indent=4)

# Print statistics
print("\n--- Filtering Results ---")
print(f"Original dataset size: {len(final_data_list)} entries")
print(f"Filtered dataset size: {len(filtered_data_list)} entries")
print(f"Removed {len(final_data_list) - len(filtered_data_list)} entries")
print(f"\nRemaining species: {sorted(filtered_class_names)}")
print(f"\nSaved filtered data to: {filtered_output_file}")
print(f"Saved filtered class names to: {FILTERED_CLASS_NAMES_JSON}")


--- Starting Ixodes Filtering ---
Found species before filtering: ['Amblyomma americanum', 'Amblyomma maculatum', 'Dermacentor andersoni', 'Dermacentor variabilis', 'Haemaphysalis longicornis', 'Ixodes', 'Ixodes cookei', 'Ixodes scapularis']

--- Filtering Results ---
Original dataset size: 1324 entries
Filtered dataset size: 1160 entries
Removed 164 entries

Remaining species: ['Amblyomma americanum', 'Amblyomma maculatum', 'Dermacentor andersoni', 'Dermacentor variabilis', 'Haemaphysalis longicornis', 'Ixodes cookei', 'Ixodes scapularis']

Saved filtered data to: ../data/processed/final_data_no_ixodes.json
Saved filtered class names to: ../data/processed/class_names_no_ixodes.json


## Getting number of Specimens in each resource. Checks for duplicates and missing datapoints.


In [47]:
# --- Step 1: Get all Sample IDs from the CSV ---
# This is our "source of truth".
df = pd.read_csv(METADATA_PATH)
csv_ids = set(df['Sample ID'].astype(str))
print(f"Found {len(csv_ids)} unique Sample IDs in the CSV file.")

# --- Step 2: Get all base IDs from the image filenames ---
# This is what's actually in our image folder.
image_base_ids = set()
for filename in os.listdir(IMAGE_DIR):
    # We split "ZOE-0013-09-01.JPG" and take the first 3 parts
    parts = filename.split('-')
    if len(parts) >= 3:
        # Rejoin to form the base ID, e.g., "ZOE-0013-09"
        base_id = '-'.join(parts[:-1]) # Takes all parts except the last one
        image_base_ids.add(base_id)

print(f"Found {len(image_base_ids)} unique base IDs from the image filenames.")

# --- Step 3: Find the Mismatches ---
# This is the most important part. We find what's in one list but not the other.

missing_from_images = csv_ids.difference(image_base_ids)
missing_from_csv = image_base_ids.difference(csv_ids)

print("\n--- DIAGNOSTIC RESULTS ---\n")

if not missing_from_images:
    print("✅ All Sample IDs from the CSV have a corresponding image base ID.")
else:
    print(f"🚨 Found {len(missing_from_images)} IDs in the CSV that have NO matching image base ID:")
    # Print the first 10 examples so we can inspect them
    print(list(missing_from_images)[:10])

print("-" * 30)

if not missing_from_csv:
    print("✅ All image base IDs have a corresponding entry in the CSV.")
else:
    print(f"🚨 Found {len(missing_from_csv)} image base IDs that are NOT in the CSV:")
    # Print the first 10 examples
    print(list(missing_from_csv)[:])

Found 665 unique Sample IDs in the CSV file.
Found 667 unique base IDs from the image filenames.

--- DIAGNOSTIC RESULTS ---

🚨 Found 1 IDs in the CSV that have NO matching image base ID:
['ZOE-0086-01']
------------------------------
🚨 Found 3 image base IDs that are NOT in the CSV:
['59-01', 'ZOE-0086-02', 'ZOE-0095-05']


## Checking how many Specimens have only 1 image

In [48]:
# We'll reuse the 'image_base_ids' logic from our diagnostic script,
# but this time we'll count the occurrences.
from collections import Counter

image_id_counts = []
for filename in os.listdir(IMAGE_DIR):
    parts = filename.split('-')
    if len(parts) >= 3:
        base_id = '-'.join(parts[:-1])
        image_id_counts.append(base_id)

# Count how many times each base_id appears
id_counts = Counter(image_id_counts)

# Now, find the IDs that don't have exactly 2 images
specimens_with_one_image = {id: count for id, count in id_counts.items() if count == 1}
specimens_with_more_than_two = {id: count for id, count in id_counts.items() if count > 2}


print("--- IMAGE COUNT ANALYSIS ---")
print(f"Total unique base IDs found: {len(id_counts)}")

if specimens_with_one_image:
    print(f"\n Found {len(specimens_with_one_image)} specimens with only ONE image:")
    print(specimens_with_one_image)
else:
    print("\n All specimens appear to have at least two images.")

if specimens_with_more_than_two:
    print(f"\n Found {len(specimens_with_more_than_two)} specimens with MORE than two images:")
    print(specimens_with_more_than_two)

--- IMAGE COUNT ANALYSIS ---
Total unique base IDs found: 667

 Found 2 specimens with only ONE image:
{'246-01': 1, '33-01': 1}

 Found 1 specimens with MORE than two images:
{'435-01': 3}


## Matching the images with the available data and building the JSON

In [49]:
# final_data_list = []
# records_with_missing_images = []

# # Iterate through each row of the metadata DataFrame.
# for index, row in df.iterrows():
#     # Get the complete identifier directly from the 'Sample ID' column.
#     base_id = row['Sample ID']

#     # Construct the expected dorsal and ventral filenames (converted to uppercase for matching).
#     dorsal_filename = f"{base_id}-01.JPG" or f"{base_id}-1.JPG"
#     ventral_filename = f"{base_id}-02.JPG"

#     # Check if both uppercase filenames exist in our set of actual filenames.
#     if dorsal_filename in all_image_filenames and ventral_filename in all_image_filenames:
        
#         # Create the dictionary for the DORSAL image.
#         dorsal_entry = {
#             'image_path': os.path.join(IMAGE_DIR, dorsal_filename),
#             'true_label': row['Species of Tick'],
#             'sample_id': row['Sample ID'],
#             'view': 'dorsal',
#             'sex': row.get('Sex'),
#             'life_stage': row.get('Life Stage'),
#             'attached': row.get('Attached?')
#         }
#         final_data_list.append(dorsal_entry)

#         # Create the dictionary for the VENTRAL image.
#         ventral_entry = {
#             'image_path': os.path.join(IMAGE_DIR, ventral_filename),
#             'true_label': row['Species of Tick'],
#             'sample_id': row['Sample ID'],
#             'view': 'ventral',
#             'sex': row.get('Sex'),
#             'life_stage': row.get('Life Stage'),
#             'attached': row.get('Attached?')
#         }
#         final_data_list.append(ventral_entry)
#     else:
#         # If one or both images are missing, log the record for review.
#         records_with_missing_images.append(base_id)

# print(f"Successfully created {len(final_data_list)} data entries for the JSON file.")
# if records_with_missing_images:
#     print(f"\nWarning: Could not find image pairs for the following {len(records_with_missing_images)} records:")
#     print(records_with_missing_images)